In [1]:
# %pylab notebook
import numpy as np 
import h5py
import matplotlib.pyplot as plt
%matplotlib ipympl

In [2]:
'''
column 1: group ID
column 2: richness
column 3: ra
column 4: dec
column 5: redshift
column 6: log M_h/(M_sun/h)
column 7: L_group (10^10Lsun/h/h)
'''

dgroup = np.loadtxt('../odata/CLAUDS_HSC_iband_group')
d2 = np.loadtxt('../odata/iCLAUDS_HSC_iband_2')
digal = np.loadtxt('../odata/CLAUDS_HSC_iband_igal')

In [3]:
def sigma2dis(m_m0,c_c0,chi2k,prisig):
    mave = np.mean(m_m0)
    cave = np.mean(c_c0)
    msigma = np.std(m_m0,ddof = 1)
    csigma = np.std(c_c0,ddof = 1)
    pcov = np.cov(m_m0,c_c0)[0][1] / msigma / csigma
    if prisig == True:
        print(np.cov(m_m0,c_c0),msigma,csigma)
        print('The correlation coefficience is %s' %pcov)
    # pos = np.zeros(shape = (m_m0.shape[0],2))
    # pos[:,0] = datafit[loc,0] / dataCM[:,2] * 1e15
    # pos[:,1] = datafit[loc,1] / dataCM[:,6]
    # cov=np.cov(np.log10(pos).T)
    # l1=plot_cov_ellipse(cov, np.log10(pos).mean(axis=0), color='green', fill=0)
    # print(np.log10(pos).mean(axis=0),mave,cave)

    def gaussian2(xx,yy):
        A = 1 / 2 / np.pi / msigma / csigma / np.sqrt(1 - pcov**2)
        B = (xx - mave)**2 / msigma**2 + (yy - cave)**2 / csigma**2 - 2*pcov*(xx - mave)*(yy - cave) / msigma / csigma
        return A * np.exp(-1 / 2 / (1 - pcov**2) * B)

    xx = np.linspace(mave-0.01,mave+0.01,400)
    yy = np.linspace(cave-0.01,cave+0.01,400)
    X,Y = np.meshgrid(xx,yy)
    Z = gaussian2(X,Y)

    k1 = np.exp(-chi2k / 2) / 2 / np.pi / msigma / csigma / np.sqrt(1 - pcov**2)
    sigmalevel = [k1]
    return X,Y,Z,sigmalevel

def plot_elp(m_m0,c_c0,chi2k):
    mave = np.mean(m_m0)
    cave = np.mean(c_c0)
    cov = np.cov(m_m0,c_c0)
    
    sig2m = cov[0,0]
    sig2c = cov[1,1]
    sigmc = cov[0,1]
    n1 = 1/2*(sig2m+sig2c+((sig2m-sig2c)**2+4*sigmc**2)**0.5) #to large eig
    n2 = 1/2*(sig2m+sig2c-((sig2m-sig2c)**2+4*sigmc**2)**0.5) #to samll eig
    vals, vecs = np.linalg.eig(cov)
    invT = np.linalg.inv(vecs)

    a = invT[0,0]
    b = invT[0,1]
    c = invT[1,0]
    d = invT[1,1]
    
    def elp(xx,yy):
        w1 = a*(xx-mave)+b*(yy-cave)
        w2 = c*(xx-mave)+d*(yy-cave)
        return w1**2/vals[0] + w2**2/vals[1]
    
    xx = np.linspace(mave-0.01,mave+0.01,400)
    yy = np.linspace(cave-0.01,cave+0.01,400)
    X,Y = np.meshgrid(xx,yy)
    Z = elp(X,Y)
    
    len1 = 2*(chi2k*n1)**0.5
    len2 = 2*(chi2k*n2)**0.5
    if len1 > len2:
        print('ratio between large and small axis',len2/len1)
    else:
        print('ratio between large and small axis',len1/len2)
    return X,Y,Z

In [4]:
def cal_axis_ratio(m_m0,c_c0,chi2k):
    cov = np.cov(m_m0,c_c0)
    
    sig2m = cov[0,0]
    sig2c = cov[1,1]
    sigmc = cov[0,1]
    n1 = 1/2*(sig2m+sig2c+((sig2m-sig2c)**2+4*sigmc**2)**0.5) #to large eig
    n2 = 1/2*(sig2m+sig2c-((sig2m-sig2c)**2+4*sigmc**2)**0.5) #to samll eig
    
    len1 = 2*(chi2k*n1)**0.5
    len2 = 2*(chi2k*n2)**0.5
    if len1 > len2:
        return len2/len1
    else:
        return len1/len2

def sample_group(id_group, redrange):
    groupid = np.int64(dgroup[id_group,0])
    Ngroup = id_group.shape[0]
    print(Ngroup)
    ax_ratio = np.zeros(Ngroup)
    for i in range(Ngroup):
        cluid = groupid[i]
        clura = dgroup[cluid-1,2]
        cludec = dgroup[cluid-1,3]
        memid = np.int64(d2[d2[:,0]==cluid,1])
        if redrange == 'low' and memid.shape[0] <= 5:
            memabmag = digal[memid,5]
            sortid = np.argsort(memabmag)[:5]
            memid = memid[sortid]
            
        memra = digal[memid-1,1]
        memdec = digal[memid-1,2]
        
        
        ax_ratio[i] = cal_axis_ratio(memra,memdec,chi2k = 2.3)
    return ax_ratio,Ngroup

## Halo mass distribution

In [5]:
redz = [3.2,3.6]
nmemdown = 5
nmemup = 5
id_group0 = np.where((dgroup[:,4] > redz[0]) & (dgroup[:,4] < redz[1]) & (dgroup[:,3] < 50)
                    & (dgroup[:,1] >= nmemdown) & (dgroup[:,1] <= nmemup))[0]
ax_ratio0,Ngrp0 = sample_group(id_group0,'high')


redz = [0,0.4]
Mherr = np.percentile(dgroup[id_group0,-2], [16,84])
Mhdown = Mherr[0]
Mhup = Mherr[1]
print("mass range is ", Mhdown, Mhup)
id_group = np.where((dgroup[:,4] > redz[0]) & (dgroup[:,4] < redz[1]) & (dgroup[:,3] < 50)
                   & (dgroup[:,-2] > Mhdown) & (dgroup[:,-2] < Mhup))[0]
ax_ratio,Ngrp = sample_group(id_group,'low')

82
mass range is  12.491700000000002 12.920371999999999
3557


/opt/anaconda-mamba/envs/mamba-python3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda-mamba/envs/mamba-python3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Degrees of freedom <= 0 for slice
  
/opt/anaconda-mamba/envs/mamba-python3/lib/python3.7/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/opt/anaconda-mamba/envs/mamba-python3/lib/python3.7/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [7]:
plt.figure(figsize = (5.3,4.0))
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 1.2 #set the value globally


plt.hist(ax_ratio0, bins = 15, alpha = 0.8, range = (0,1), 
         label = '%s < $z$ < %s' %(3.2,3.6), density = True)

plt.hist(ax_ratio, bins = 15, alpha = 1, range = (0,1), histtype = 'step', ls = '--', color = 'r', lw = 2,
         label = '%s < $z$ < %s' %(0,0.4), density = True)

plt.legend(loc = 2)
plt.xlim(0,1)
plt.xlabel(r'$\gamma_{\rm ab}$', fontsize = 14)
plt.tick_params(top = 'on', right = 'on', which='both', direction = 'in', labelsize = 12)
plt.ylabel(r'$\rm PDF$', fontsize = 14)

plt.tight_layout()
plt.savefig('../figs/gamma.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
# redz = [0.0,0.4]
# nmemdown = 4
# nmemup = 8
# Mhdown = 12.15
# Mhup = 12.46
# id_group = np.where((dgroup[:,4] >= redz[0]) & (dgroup[:,4] < redz[1]) 
#                     & (dgroup[:,1] >= nmemdown) & (dgroup[:,1] <= nmemup))[0]

# plt.figure()

# plt.hist(dgroup[id_group,1])
# print(np.max(dgroup[id_group,1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

8.0


In [ ]:
# plt.hist(ax_ratio, bins = 15, alpha = 0.3, range = (0,1), 
#          label = '%s < z < %s (%s)' %(redz[0],redz[1],Ngrp), weights = np.tile(1/Ngrp,Ngrp))